# Submission for Coursework 3
Team: Billie-Jo Powers, Jasmine Burgess, $\tiny{Harry}$ & Mark Holcroft 

In [ ]:
# imports 
import sys
import pickle
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
sys.path.append('..')

## Introduction

This will contain a small exposition of what we are hoping to do. Probably going to wait until this is done in the report to drag and drop in here. 

## Simplified Game of Pig

some exposition about the game of pig and why it is simpler. Say that we are about to make the same thing as figure 2 in the paper. 

In [ ]:
from piglet import PigletSolver
piglet_plotter = PigletSolver(goal = 2, epsilon = 10e-6)
piglet_plotter(convergence_plots=True)

talk about how early states have to wait for later ones to converge. Hence this motivates other procedure. 

## Surface Plot of Optimal Policy

we now apply this to the next section which is the full game of pig. This starts by remaking plots 3 from the paper. 

In [ ]:
# Due to pre complilation of loops using numba package, the optimal policy can be derived in minimal time
from notebook_writeup.optimised_layered_vi import pig_layered_value_iteration
 
die_size = 6
target_score = 100
max_turn = 100
values, policy = pig_layered_value_iteration(target_score=target_score, 
                                             die_sides=die_size, 
                                             max_turn=max_turn, 
                                             epsilon=1e-6)

In [ ]:
# We can display our policy using the following plotting module
from notebook_writeup.plotting_tools import surface_plotter
surface_plotter(policy)

### Extracting Reachable States 

this is most easily done by exact simulation in my opinion. Thats how i did it, and thats the file I read in here. 

In [ ]:
# reachable states are extractable from experiment
with open('pickle_and_config_files/reachable_states.pkl', 'rb') as d:
    reachable_states = pickle.load(d)
surface_plotter(reachable_states)

below we will have a cross section of this which is meant to look like the one from the lecture 

In [ ]:
cross_section_reachable = np.array([[reachable_states[(i, 30, k)] for i in range(101)] for k in range(100, -1, -1)])
cross_section_policy = policy[:, 30, ::-1]

In [ ]:
plt.figure(figsize=(8, 8))

# Show the two imshow layers
plt.imshow(cross_section_policy.T, cmap='Blues', alpha=0.6)
plt.imshow(cross_section_reachable, cmap='Reds', alpha=0.4)

# Axis labels
plt.xlabel("Player Score")
plt.ylabel("Opponent Score")

# Custom legend using colored patches
policy_patch = mpatches.Patch(color='blue', alpha=0.6, label='Policy Region')
reachable_patch = mpatches.Patch(color='red', alpha=0.4, label='Reachable Region')
plt.legend(handles=[policy_patch, reachable_patch], loc='upper right')

plt.title("Policy vs Reachable State Overlay")

plt.show()


notes on the above plot: I think it looks pretty shit. I will be rerunning the code which maps the state space for many more iterations. Hopefully that will give us a lot more smoothness on the red region

### Contours of winning probability

Small talk about what this is etc 

In [ ]:
# make use of our final values outputted by the 
from notebook_writeup.plotting_tools import plot_isosurface_from_array

plot_isosurface_from_array(values, isovalues=[0.03, 0.09, 0.27, 0.81])

### Reproducing Score Analytics 

what a shit title that defo needs to change btw, but that is what we settle on for now 

I must also confess that i changed your code to make it so the competition code will read from numpy arrays rather than the dictionaries. Bit of a pisstake i know, but the pickle dicts served their time well during development

In [ ]:
# In section 'the solution' they rattle off some %s for wins and stuff. we will try and reproduce them here 
from competition import Competition
competition_function = Competition(policy, policy, replications=100000, seed = 1)
win_percentage = competition_function()
print(f'For two optimal policies playing against each other, player 1 wins {round(win_percentage*100, 3)}% of the time.')

In [ ]:
# note that this win percentage is the same as given in the value function
float(values[0,0,0] * 100)

next we want something like this but for playing against the hold at 20 player. 

In [ ]:
# we have another class called opponents which can make policies for us to play against if we want to 
from competition import Opponents
hold_at_20_policy = Opponents.hold_at_20()

In [ ]:
competition_function_h20 = Competition(policy, hold_at_20_policy, replications=100000, seed = 1)
win_percentage_h20 = competition_function_h20()
print(f'For optimal policy playing against hold at 20, optimal player wins {round(win_percentage_h20*100, 3)}% of the time.')